In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch

print("🚀 Starting SDG Classification (Unsupervised AI)...")

# 1. Load Data
# ใช้ไฟล์ที่มี Q1 ก็ได้ครับ ข้อมูลจะได้ครบๆ
try:
    df = pd.read_csv('papers_all_years.csv')
except:
    df = pd.read_csv('papers_all_years.csv')

# เลือกเฉพาะที่มี Title
df = df[['title', 'year', 'journal', 'subject_areas_str']].dropna(subset=['title'])
print(f"   -> Loaded {len(df)} papers")

print("🚀 Starting SDG Classification (Unsupervised AI)...")

# 1. Load Data & Clean Up
try:
    df = pd.read_csv('papers_all_years.csv')
    print(f"   -> Original CSV rows: {len(df)}") # เช็คยอดหลังอ่านทันที
except:
    print("❌ Error: File not found")
    exit()

# เลือกคอลัมน์และลบค่าว่าง
df = df[['title', 'year', 'journal', 'subject_areas_str']].dropna(subset=['title'])

# --- [เพิ่มส่วนนี้] แก้ปัญหาแถวงอก ---
# 1. ลบแถวที่ Title ซ้ำกัน (Deduplicate)
df = df.drop_duplicates(subset=['title'])

# 2. ลบแถวที่ Title สั้นผิดปกติ (อาจเป็นเศษบรรทัดที่หลุดมา)
df = df[df['title'].str.len() > 5] 

# 3. สำคัญมาก! รีเซ็ต Index ไม่งั้นตอนแปะคำตอบจะ Error
df = df.reset_index(drop=True) 

print(f"   -> Cleaned rows to process: {len(df)}") # ยอดนี้ควรจะ <= 20217

# 2. Define SDGs (นิยามเป้าหมาย 17 ข้อ เพื่อให้ AI เข้าใจ)
sdg_definitions = {
    'SDG 1': 'No Poverty: End poverty in all its forms everywhere, economic growth, social protection.',
    'SDG 2': 'Zero Hunger: End hunger, achieve food security and improved nutrition and promote sustainable agriculture.',
    'SDG 3': 'Good Health and Well-being: Ensure healthy lives and promote well-being for all at all ages, medicine, disease, virus, mental health.',
    'SDG 4': 'Quality Education: Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.',
    'SDG 5': 'Gender Equality: Achieve gender equality and empower all women and girls.',
    'SDG 6': 'Clean Water and Sanitation: Ensure availability and sustainable management of water and sanitation for all.',
    'SDG 7': 'Affordable and Clean Energy: Ensure access to affordable, reliable, sustainable and modern energy for all, electricity, renewable.',
    'SDG 8': 'Decent Work and Economic Growth: Promote sustained, inclusive and sustainable economic growth, full and productive employment.',
    'SDG 9': 'Industry, Innovation and Infrastructure: Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation.',
    'SDG 10': 'Reduced Inequalities: Reduce inequality within and among countries.',
    'SDG 11': 'Sustainable Cities and Communities: Make cities and human settlements inclusive, safe, resilient and sustainable, urban planning.',
    'SDG 12': 'Responsible Consumption and Production: Ensure sustainable consumption and production patterns, waste management, recycling.',
    'SDG 13': 'Climate Action: Take urgent action to combat climate change and its impacts, global warming, carbon emission.',
    'SDG 14': 'Life Below Water: Conserve and sustainably use the oceans, seas and marine resources, marine biology.',
    'SDG 15': 'Life on Land: Protect, restore and promote sustainable use of terrestrial ecosystems, forests, biodiversity.',
    'SDG 16': 'Peace, Justice and Strong Institutions: Promote peaceful and inclusive societies, justice, law.',
    'SDG 17': 'Partnerships for the Goals: Strengthen the means of implementation and revitalize the global partnership.'
}

sdg_labels = list(sdg_definitions.keys())
sdg_texts = list(sdg_definitions.values())

# 3. Load AI Model (Sentence-BERT)
# โมเดลนี้จะแปลงประโยคเป็น Vector 384 มิติ
print("🤖 Loading AI Model (all-MiniLM-L6-v2)... This might take a minute.")
model = SentenceTransformer('all-MiniLM-L6-v2')

# 4. Encoding (แปลงข้อความให้เป็นคณิตศาสตร์)
print("⚙️ Embedding SDG Definitions...")
sdg_embeddings = model.encode(sdg_texts, convert_to_tensor=True)

print("⚙️ Embedding Research Titles (This identifies the meaning)...")
paper_embeddings = model.encode(df['title'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# 5. Semantic Search (หาว่า Paper นี้ตรงกับ SDG ข้อไหนที่สุด)
print("🔍 Matching Papers to SDGs...")
# คำนวณ Cosine Similarity ระหว่าง Paper ทุกใบ กับ SDG ทุกข้อ
cosine_scores = util.cos_sim(paper_embeddings, sdg_embeddings)

# หาค่าสูงสุด (Top Match)
top_results = torch.topk(cosine_scores, k=1, dim=1)
top_indices = top_results.indices.flatten().tolist()
top_scores = top_results.values.flatten().tolist()

# บันทึกผลลง DataFrame
df['Predicted SDG'] = [sdg_labels[i] for i in top_indices]
df['SDG Score'] = top_scores # ค่าความมั่นใจของ AI

# กรอง: ถ้า Score น้อยกว่า 0.25 อาจจะไม่เกี่ยวกับ SDG เลย ให้เป็น "General Research"
threshold = 0.25
df.loc[df['SDG Score'] < threshold, 'Predicted SDG'] = 'General / Non-SDG'

print("✅ Classification Complete!")
print(df['Predicted SDG'].value_counts().head())

# 6. Save
df.to_csv('chula_sdg_classified.csv', index=False)
print("💾 Saved to 'chula_sdg_classified.csv'")

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "C:\Users\สึนามิ\AppData\Roaming\Python\Python312\site-packages\torch\lib\c10.dll" or one of its dependencies.